The goal of this project is to use the pretrained RoBERTa transformer as a feature extractor with a costum classification head to determine if text messages are offensive or not.

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoConfig

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import seaborn as sns
import matplotlib.pyplot as plt

import wandb

import sys
from pathlib import Path

# Add src/ to path (once, so imports work)
sys.path.append(str(Path().resolve().parent / "src"))

%load_ext autoreload
%autoreload 2
from helper_functions import train_model, test_model, get_class_distribution, oversample_dataset, undersample_dataset, AttentionPooling

In [2]:
from paths import DATA_CLEANED, DATA_PROCESSED
print("Cleaned data path:", DATA_CLEANED)
print("Processed data path:", DATA_PROCESSED)

Cleaned data path: /Project/data/cleaned
Processed data path: /Project/data/processed


In [3]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Using RoBERTa as a feature extractor with a costum classification head

Found this pretrained model online: cardiffnlp/twitter-roberta-base-sentiment-latest (https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest)

It is already pretrained on twitter messages. 

Define which pretrained model is used and initilise tokenizer

In [4]:
model_name = 'roberta-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)

Classification head

In [5]:
class CustomClassifier(nn.Module):
    def __init__(self, model_name, class_weights_tensor, pooling='cls'):
        super().__init__()
        self.class_weights = class_weights_tensor
        self.pooling = pooling.lower()
        self.base = AutoModel.from_pretrained(model_name)
        config = AutoConfig.from_pretrained(model_name)
        hidden_size = config.hidden_size  # Dynamically get the model's hidden size

        if pooling == 'attention_pooling':
            # Replace CLS pooling with attention
            self.attention_pool = AttentionPooling(hidden_size)
        
        # Freeze all parameters of the base model
        for param in self.base.parameters():
            param.requires_grad = False

        # Custom classification head
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.LeakyReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 2)
        )

    def forward(self, input_ids, attention_mask, class_weights_tensor = None, labels=None):
        outputs = self.base(input_ids=input_ids, attention_mask=attention_mask)

        # Pooling strategy: either CLS token or mean pooling over token embeddings
        if self.pooling == 'mean':
            token_embeddings = outputs.last_hidden_state
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
            sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
            sum_mask = input_mask_expanded.sum(1).clamp(min=1e-9)
            pooled = sum_embeddings / sum_mask
        elif self.pooling == 'attention_pooling':
            pooled = self.attention_pool(outputs.last_hidden_state, attention_mask)
        else:
            pooled = outputs.last_hidden_state[:, 0, :]  # CLS token

        logits = self.classifier(pooled)

        # If labels are provided, calculate the loss
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor.to(device))
            loss = loss_fn(logits, labels)
            return logits, loss

        return logits

## Load HASOC dataset for training, validation and testing

In [6]:
# Custom Dataset Class
class HateSpeechDataset(Dataset):
    def __init__(self, df, tokenizer, label = 'label', max_len=128):
        self.texts = df["text"].tolist()
        self.labels = df[label].tolist()
        self.encodings = tokenizer(self.texts, padding=True, truncation=True, max_length=max_len)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

Define experiment scope:

In [7]:
# here we are just using the labels of the first task of the HASOC dataset, which is a binary classification
label = "task_1"

In [8]:
# Load training and test data
clean_df = pd.read_csv(DATA_CLEANED / "hasoc_2019_en_train_cleaned.tsv", sep='\t')
# test_df = pd.read_csv(DATA_PROCESSED / "hasoc_2019_en_test.tsv", sep='\t')
test_df = pd.read_csv(DATA_CLEANED / "hasoc_2019_en_test_cleaned.tsv", sep='\t')

# Split clean dataset in training and validation set
train_df, val_df = train_test_split(clean_df, test_size=0.3, random_state=42, stratify=clean_df[label])

# Automatically map string labels to integers
label_list = sorted(train_df[label].unique())
label_map = {label: idx for idx, label in enumerate(label_list)}

train_df[label] = train_df[label].map(label_map)
val_df[label] = val_df[label].map(label_map)
test_df[label] = test_df[label].map(label_map)

In [9]:
# Decide which technique to use to cope with data imbalance
handling_imbalance = "class_weighting"
# when choosing 'class_weighting' dataset is not touched but classes gets weighted depending on label/class distribution

if handling_imbalance == 'oversampling':
    # Oversample dataset
    train_df = oversample_dataset(train_df, label)
    # val_df = oversample_dataset(val_df, label) # over and undersampling only useful for training dataset
    # test_df = oversample_dataset(test_df, label)
elif handling_imbalance == 'undersampling':
    # Undersample dataset
    train_df = undersample_dataset(train_df, label)
    # val_df = undersample_dataset(val_df, label)
    # test_df = undersample_dataset(test_df, label)

In [10]:
# Create PyTorch Datasets and DataLoaders
train_dataset = HateSpeechDataset(train_df, tokenizer, label=label)
val_dataset = HateSpeechDataset(val_df, tokenizer, label=label)
test_dataset = HateSpeechDataset(test_df, tokenizer, label=label)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=32)

print(get_class_distribution(train_df, label))

{1: 2513, 0: 1583}


In [11]:
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_df[label]),
    y=train_df[label]
)

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

## Training and evaluation of model

In [12]:
# Decide what pooling to use
pooling = "attention_pooling"

# Initialize model
model = CustomClassifier(model_name, class_weights_tensor, pooling=pooling).to(device)

# Optimizer only for the classification head
optimizer = torch.optim.AdamW(model.classifier.parameters(), lr=2e-4)
epochs = 100

2025-05-14 13:57:45.643272: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-14 13:57:45.658922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747223865.678549   60513 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747223865.684633   60513 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747223865.699093   60513 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [13]:
wandb.init(project="roberta-classifier", config={
    "model": model_name,
    "frozen_base": True,
    "pooling": pooling,
    "epochs": epochs,
    "lr": 1e-5,
    "handling_imbalance": handling_imbalance
})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: natalia-timokhova-v (natalia-timokhova-v-lule-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
train_model(model, train_loader, val_loader, optimizer, device, epochs)


Epoch 1
Training Loss: 0.6683
Train Accuracy: 0.5886
Train F1 (macro): 0.5758

Val Loss: 0.6482
Val Accuracy: 0.6207
Val F1 (macro): 0.6128

New best model saved (F1: 0.6128, Acc: 0.6207)

Epoch 2
Training Loss: 0.6364
Train Accuracy: 0.6340
Train F1 (macro): 0.6231

Val Loss: 0.6366
Val Accuracy: 0.6333
Val F1 (macro): 0.6289

New best model saved (F1: 0.6289, Acc: 0.6333)

Epoch 3


## Testing of model

In [ ]:
# Load best model
model.load_state_dict(torch.load("best_model.pt", weights_only=True))
# Test model on test set
test_model(model, test_loader, device, phase = "test")